In [ ]:
'''
PURPOSE: 
Download images into folder
    images/[actor]/[actor]_processed
Process and save download images to folder
    images/[actor]/[actor]_unprocessed
create dataset: x, y
'''

In [1]:
import urllib
import os
import numpy as np
import pandas as pd
from random import randrange
import sys
from scipy.misc import imread, imshow, imresize, imsave
import matplotlib.pyplot as plt

In [2]:
def timeout(func, args=(), kwargs={}, timeout_duration=1, default=None):
    '''From:
    http://code.activestate.com/recipes/473878-timeout-function-using-threading/'''
    import threading
    class InterruptableThread(threading.Thread):
        def __init__(self):
            threading.Thread.__init__(self)
            self.result = None

        def run(self):
            try:
                self.result = func(*args, **kwargs)
            except:
                self.result = default

    it = InterruptableThread()
    it.start()
    it.join(timeout_duration)
    if it.isAlive():
        return False
    else:
        return it.result

In [3]:
def rgb2gray(rgb):
    '''Return the grayscale version of the RGB image rgb as a 2D numpy array
    whose range is 0..1
    Arguments:
    rgb -- an RGB image, represented as a numpy array of size n x m x 3. The
    range of the values is 0..255
    '''
    
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray/255.

In [4]:
def process_img(image, coords):
    image_cropped = image[coords[1]:coords[3], coords[0]:coords[2], :]
    image_gray  = rgb2gray(image_cropped)
    processed_image = imresize(image_gray, (32,32))
    
    return processed_image

In [5]:
def to_list(coord_string):
    return map(int,coord_string.split(","))


In [6]:
act = list(set([a.split("\n")[0] for a in open("not_subset_actors.txt").readlines()]))

In [7]:
#create dataframe of actor names, URL and coordinates of bounding boxes
facescrub_acts = pd.read_table("all_acts.txt",sep="\t",header=None)
facescrub_acts.columns = ["Name", "Col1", "Col2", "URL", "Coordinates", "Col5"]
facescrub_acts_trimmed = facescrub_acts.drop(facescrub_acts.columns[[1,2,5]], axis = 1)

In [8]:
#create directory for unprocessed images
current_directory = os.getcwd()
images = os.path.join(current_directory, r'images')
if not os.path.exists(images):
   os.makedirs(images)

#create directory for each actor
for actor in act:
    actor_directory = os.path.join(images,actor)
    if not os.path.exists(actor_directory):
        os.makedirs(actor_directory)
        os.makedirs(os.path.join(actor_directory,actor+r'_unprocessed'))
        os.makedirs(os.path.join(actor_directory,actor+r'_processed'))


In [13]:
#download and process images and create dataset
#probably should stop this once I've downloaded 100 images ... -_-

i=0
for index, row in facescrub_acts_trimmed.iterrows():
    actor = row['Name']
    if actor in act:
          
        image_name = actor+'_{}.jpg'.format(i)
        unprocessed_dir = os.path.join(images,actor,actor+r'_unprocessed')
        processed_dir = os.path.join(images,actor, actor+r'_processed')
        image_name_w_path = os.path.join(unprocessed_dir,image_name)
        processed_image_w_path = os.path.join(processed_dir,image_name)
        
        if (row['URL'] == "http://www.sawfirst.com/wp-content/uploads/20"):
            #one awkward link that asks for a password...
            print("Awkward link")
        else:
            try:
                #don't download images that take longer than 2 seconds (I pay for BELL's Fibe300)
                timeout(urllib.urlretrieve, (row['URL'],image_name_w_path),{}, 2)
                image= imread(image_name_w_path) #check to see if image actually opens, if it doesn't handle exception
                image_processed = process_img(image, to_list(row['Coordinates'])) 
                imsave(processed_image_w_path, image_processed)

                #create data set
                if i == 0:
                    x = image_processed
                    y = np.array([actor])
                else: 
                    x = np.dstack((x, image_processed))
                    y = np.append(y,[actor])

                i+=1 #if image file doesn't open, i won't be incremented
                print(image_name_w_path)
                print("working:     "+row['URL'])

            except Exception:
                #if url is broken
                sys.exc_clear()
                print("not working:     "+row['URL'])


/Users/Admin/anaconda3/envs/CSC411_py2_env/lib/python2.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/Users/Admin/anaconda3/envs/CSC411_py2_env/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  after removing the cwd from sys.path.
/Users/Admin/anaconda3/envs/CSC411_py2_env/lib/python2.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_0.jpg
working:     http://collider.com/wp-content/uploads/daniel-radcliffe-horns.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_1.jpg
working:     http://images4.fanpop.com/image/photos/22300000/Daniel-Radcliffe-Drama-Desk-Awards-daniel-radcliffe-22361539-909-1222.jpg
not working:     http://brightmosaicminds.com/wp-content/uploads/2013/11/Daniel-Radcliffe_2.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_2.jpg
working:     http://images6.fanpop.com/image/photos/35400000/dan-daniel-radcliffe-35454992-957-1200.jpg
not working:     http://shechive.files.wordpress.com/2011/04/eye-candy-daniel-radcliffe-26.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_

not working:     http://stylefavor.com/wp-content/uploads/2013/01/daniel_radcliffe-wallpapers.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_25.jpg
working:     http://i2.cdnds.net/13/02/618x847/daniel_radcliffe.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_26.jpg
working:     http://cdn03.cdn.justjared.com/wp-content/uploads/2011/06/radcliffe-trevor/daniel-radcliffe-trevor-live-07.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_27.jpg
working:     http://s1.ibtimes.com/sites/www.ibtimes.com/files/styles/picture_this/public/2011/07/05/124753-actor-daniel-radcliffe.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_28

/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_53.jpg
working:     http://images5.fanpop.com/image/photos/26200000/Daniel-Radcliffe-Wallpaper-The-Woman-In-Black-daniel-radcliffe-26273249-1024-768.png
not working:     http://www.hairstylepedia.com/images/02/Daniel-Radcliffe-with-short-layered-hair.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_54.jpg
working:     http://www.andpop.com/wp-content/uploads/2010/03/daniel-radcliffe-harry-potter1.jpg
not working:     http://www.eonline.com/eol_images/Entire_Site/2013119/rs_634x1024-131209090054-634.daniel-radcliffe-graham-norton.ls.12913.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_55.jpg
working:     http://images5.fanpop.com/image/photos/26200000/Daniel-Radcliff

/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_78.jpg
working:     http://images4.fanpop.com/image/photos/18000000/Dan-daniel-radcliffe-18042788-1024-768.jpg
not working:     http://icydk.com/wp-content/uploads/2008/09/daniel_radcliffe_details_mag.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_79.jpg
working:     http://marieclaire.media.ipcdigital.co.uk/11116/000049c28/796e_orh100000w440/Daniel-Radcliffe-LP3.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_80.jpg
working:     http://www.sheclick.com/wp-content/uploads/2011/03/Daniel-Radcliffe-Pic.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/Daniel Radcliffe/Daniel Radcliffe_unprocessed/Daniel Radcliffe_81.jpg
working:     http:

...

/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_926.jpg
working:     http://cdn04.cdn.justjared.com/wp-content/uploads/2010/08/ferrera-miami/america-ferrera-ryan-piers-williams-wilmer-valderrama-dry-land-miami-05.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_927.jpg
working:     http://cdn04.cdn.justjared.com/wp-content/uploads/2007/03/america_ferrera_naacp_awards_2007/america-ferrera-naacp-awards-2007-07.jpg
not working:     http://www.therichest.com/wp-content/uploads/2011/05/America-Ferrera.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_928.jpg
working:     http://img001.lazygirls.info/people/america_ferrera/america_ferrera_84_on_maxim_hot_100_ZDqSlwN.sized.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC

/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_954.jpg
working:     http://www.contactmusic.com/pics/lb/saga_12_260109/america_ferrera_2262151.jpg
not working:     http://ml.berkeleyblogs.com/archives/americaferreraaward1.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_955.jpg
working:     http://images1.fanpop.com/images/photos/2300000/Seventeen-magazine-america-ferrera-2339101-240-312.jpg
not working:     http://media.celebrity-pictures.ca/Celebrities/America-Ferrera/America-Ferrera-1093963.jpg
not working:     http://www.releasedonkey.com/big//MV5BMTQyMDUzOTkxNF5BMl5BanBnXkFtZTcwNjMwNjk0OA/america-ferrera-large-picture.jpg
not working:     http://www.hairstylescity.com/images/america-ferrera-5/America-Ferrera-11223425.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Fer

/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_982.jpg
working:     http://images.fanpop.com/images/image_uploads/America-Ferrera-america-ferrera-158501_386_425.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_983.jpg
working:     http://images1.fanpop.com/images/image_uploads/America-america-ferrera-887541_787_1222.jpg
not working:     http://thenevergirl.com/wp-content/uploads/GripEx/Sweaters/america-ferrera-sweat_10.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_984.jpg
working:     http://st.gdefon.ru/wallpapers_original/s/68419_amerika-ferrera_or_america-ferrera_1600x1200_%28www.GdeFon.ru%29.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/Ame

not working:     http://image.qpicture.com/image/a/artist-america-ferrera/america-ferrera-372039.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_1010.jpg
working:     http://i.perezhilton.com/wp-content/uploads/2011/10/america-ferrera-chicago-bro__oPt.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_1011.jpg
working:     http://www.beautyandthedirt.com/wp-content/uploads/2010/03/America-Ferrera.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_1012.jpg
working:     http://img2.timeinc.net/instyle/images/2007/galleries/070607_ferrera_400X400.jpg
not working:     http://www.hdwpapers.com/walls/america_ferrera_wallpaper_8-normal5.4.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrer

/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_1038.jpg
working:     http://www.peoples.ru/art/cinema/actor/america_ferrera/ferrera_4.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_1039.jpg
working:     http://images2.fanpop.com/images/photos/8400000/America-Ferrera-america-ferrera-8484176-300-400.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_1040.jpg
working:     http://s3.amazonaws.com/ths_assets_production/attachment_resources/attachments/1190/original/The_2007_Hot_in_Hollywood_Party_America_Ferrera.jpg
/Users/Admin/Desktop/Y3S2/CSC411 - Machine Learning/CSC411_Project1/images/America Ferrera/America Ferrera_unprocessed/America Ferrera_1041.jpg
working:     http://imworld.aufeminin.com/breves/D20111124/america-ferr

In [ ]:
#save dataset
np.save(os.path.join(os.getcwd(),"x_other_actors.npy") ,x)
np.save(os.path.join(os.getcwd(),"y_other_actors.npy") ,y)
